In [1]:
import pandas as pd
import numpy as np

In [2]:
# Leer la fuente de datos
s_ruta = r"C:\ARP\REPORTES\PRODUCCION\Produccion.feather"
df_Produccion = pd.read_feather(s_ruta)

In [3]:
# Definir los periodos
def df_Semanal(df,semana,year):
    df['SEMANA'] = df['FECHA'].dt.isocalendar().week - 1 + (df['FECHA'].dt.weekday >=5).astype(int)
    df_filtrado = df[(df['SEMANA'] == semana) & (df['FECHA'].dt.year == year)]

    return df_filtrado


In [4]:
df_Xsemana = df_Semanal(df_Produccion,14,2025)

In [5]:
i = 0
Totales = {}
Titulos = ['INGRESO REAL','COSTO REAL','INGRESO PLANEADO','COSTO PLANEADO','INGRESOS SIN PLAN','COSTOS SIN PLAN',
          'INGRESOS CON PLAN','COSTOS CON PLAN','TRABAJOS PLANEADOS','TRABAJOS NO EJECUTADOS',
          'TRABAJOS SIN PLAN']
for x in df_Xsemana[['R_INGRESO','R_COSTO','P_INGRESO','P_COSTO']].sum():
    Totales[Titulos[i]] = x
    i += 1

Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['PLANEADO'] == 'NO')]['R_INGRESO'].sum()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['PLANEADO'] == 'NO')]['R_COSTO'].sum()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['PLANEADO'] == 'SI')]['R_INGRESO'].sum()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['PLANEADO'] == 'SI')]['R_COSTO'].sum()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['R_PERSONAS'] > 0) & (df_Xsemana['PLANEADO'] == "SI")]['PLANEADO'].count()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['R_PERSONAS'] == 0) & (df_Xsemana['PLANEADO'] == "SI")]['PLANEADO'].count()
i += 1
Totales[Titulos[i]] = df_Xsemana[(df_Xsemana['PLANEADO'] == "NO")]['PLANEADO'].count()

## REPORTE SEMANAL 14 DEL 05 AL 11 DE ABRIL
El reporte de producción presenta los ingresos, los costos y la utilidad que se generan en la semana.
Presenta la información de resultados del plan de producción y su impacto en los costos de producción.\
La información que se presenta tiene como objetivo identificar las áreas de oportunidad y crear un plan de producción eficaz.
___
> #### TOTAL DE LOS TRABAJOS REALIZADOS

In [6]:
no_sem = df_Xsemana['SEMANA'].max()
utilidad = Totales['INGRESO REAL'] - Totales['COSTO REAL']
margen = utilidad / Totales['INGRESO REAL']
print(f"""SEMANA {no_sem}\n
UTILIDAD:\t$ {utilidad:,.2f}
MARGEN:\t\t{margen:.1%}\n
INGRESO:\t${Totales['INGRESO REAL']:,.2f}
COSTO:\t\t$ {Totales['COSTO REAL']:,.2f}\n""")

SEMANA 14

UTILIDAD:	$ 80,091.04
MARGEN:		56.1%

INGRESO:	$142,720.37
COSTO:		$ 62,629.33



___
### INDICADORES PARA MEDIR LA EFICACIA DEL PLAN DE PRODUCCIÓN
Tomando los datos de los trabajos planeados y los no planeados se calculan la utilidad y los margenes en cada caso.\
Con estos datos durante un periodo de tiempo podremos evaluar el impacto que tiene la improvisación, es decir el realizar trabajos no planeados, en los ingresos por utilidad bruta.
> ####  TRABAJOS REALIZADOS INCLUIDOS EN EL PLAN DE PRODUCCIÓN

In [7]:
utilidad_CP = Totales['INGRESOS CON PLAN'] - Totales['COSTOS CON PLAN']
margen_CP = utilidad_CP / Totales['INGRESOS CON PLAN']
print(f"""UTILIDAD:\t$ {utilidad_CP:,.2f}
MARGEN:\t\t{margen_CP:.1%}\n
INGRESO:\t${Totales['INGRESOS CON PLAN']:,.2f}
COSTO:\t\t$ {Totales['COSTOS CON PLAN']:,.2f}
""")

UTILIDAD:	$ 78,085.30
MARGEN:		57.3%

INGRESO:	$136,198.97
COSTO:		$ 58,113.67



> ####  TRABAJOS REALIZADOS NO INCLUIDOS EN EL PLAN DE PRODUCCIÓN

In [8]:
utilidad_SP = Totales['INGRESOS SIN PLAN'] - Totales['COSTOS SIN PLAN']
margen_SP = utilidad_SP / Totales['INGRESOS SIN PLAN']
print(f"""UTILIDAD:\t$  {utilidad_SP:,.2f}
MARGEN:\t\t{margen_SP:.1%}\n
INGRESO:\t$  {Totales['INGRESOS SIN PLAN']:,.2f}
COSTO:\t\t$  {Totales['COSTOS SIN PLAN']:,.2f}
""")

UTILIDAD:	$  2,005.73
MARGEN:		30.8%

INGRESO:	$  6,521.40
COSTO:		$  4,515.67

